In [24]:
import re
import openai
import json
import os
import pandas as pd
from google.colab import userdata
from tabulate import tabulate

# Function to initialize the chatbot conversation with a system message
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    system_message = (
        "Welcome to the India Travel Planner!\n"
        "I can help you plan your travel. Just tell me about your trip in a natural way, and I'll handle the details.\n"
        "For example, you can say 'I need to travel from Mumbai to Delhi by flight in business class' or 'Plan a road trip from Bangalore to Goa by SUV'.\n"
        "Let's begin!"
    )

    return [{"role": "system", "content": system_message}]

# Function to extract travel details from user input using OpenAI API
def dictionary_present(response):
    """
    Extracts a travel details dictionary from the user input using OpenAI API.
    """
    prompt = ("""
        You are a Python expert. Extract travel details from the given input.
        Identify and extract 'Travel_From', 'Travel_To', 'Travel_Mode', and 'Class' if present.
        Return only a JSON dictionary containing these details. No extra text.
        Recognize synonyms like 'fly' for 'Flight', 'bus ride' for 'Bus', etc.
        Example format:
        {
            "Travel_From": "Mumbai",
            "Travel_To": "Delhi",
            "Travel_Mode": "Flight",
            "Class": "Economy"
        }
        If details are missing, return an empty dictionary {}.
    """)

    messages = [{"role": "system", "content": prompt},
                {"role": "user", "content": f"Here is the user input: {response}"}]

    confirmation = get_chat_completions(messages)

    try:
        extracted_data = json.loads(confirmation["response"])
        if isinstance(extracted_data, dict):
            print("Debug: Extracted Travel Details -", extracted_data)  # Debug print
            return extracted_data
    except json.JSONDecodeError:
        print("Debug: Failed to parse extracted data.")  # Debug print
        return {}  # Return an empty dictionary if parsing fails

# Function to perform moderation check on user input
def moderation_check(user_input):
    """
    Perform moderation check on user input using the OpenAI API.
    """
    openai.api_key = userdata.get('OPENAI_API_KEY')
    response = openai.Moderation.create(input=[user_input])
    return "Flagged" if response["results"][0]["flagged"] else "Not Flagged"

# Function to get chat completions from OpenAI
def get_chat_completions(conversation, json_format=True):
    """
    Generate chat completions using OpenAI's GPT-3.5-turbo model.
    """
    openai.api_key = userdata.get('OPENAI_API_KEY')
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation
    )

    output = response.choices[0].message.content

    return {"response": output} if json_format else output

# Function to confirm that all necessary travel details are present
def intent_confirmation_layer(travel_details, conversation):
    """
    Evaluates if all necessary travel details are captured. If details are missing, asks the user naturally using GPT.
    """
    required_keys = ["Travel_From", "Travel_To", "Travel_Mode", "Class"]
    missing_details = [key for key in required_keys if key not in travel_details or not travel_details[key]]

    if not missing_details:
        return {"result": "Yes"}

    assistant_prompt = (
        f"It seems some details are missing. Please provide the missing details: {', '.join(missing_details)}"
    )

    conversation.append({"role": "assistant", "content": assistant_prompt})
    assistant_response = get_chat_completions(conversation)

    return {"result": "No", "assistant_response": assistant_response["response"]}

# Function to find matching travel options based on extracted details from a CSV file
def find_matching_travel_options(travel_details, csv_file):
    """
    Reads the CSV file and finds matching travel options based on the extracted travel details.
    """
    try:
        df = pd.read_csv(csv_file)

        matched_rows = df[
            (df["Travel_From"].str.lower() == travel_details["Travel_From"].lower()) &
            (df["Travel_To"].str.lower() == travel_details["Travel_To"].lower()) &
            (df["Travel_Mode"].str.lower() == travel_details["Travel_Mode"].lower())
        ]

        if matched_rows.empty:
            return "No matching travel options found."

        return tabulate(matched_rows.drop(columns=['Unnamed: 0'], errors='ignore'), headers='keys', tablefmt='grid')
    except Exception as e:
        return f"Error reading travel options: {str(e)}"

# Main function to run the chatbot
def main():
    print("Welcome to the India Travel Planner Chatbot!")
    print("Tell me about your trip, and I'll take care of the rest.")

    conversation = initialize_conversation()
    csv_file = "/content/drive/MyDrive/TravelPlanner/synthetic_travel_data.csv" # Mention the path of the csv data file

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Safe travels! Goodbye!")
            break

        if moderation_check(user_input) == "Flagged":
            print("Chatbot: Sorry, but I can't process that request.")
            continue  # Do not break, just ignore flagged input

        travel_details = dictionary_present(user_input)

        if not travel_details:
            print("Chatbot: I didn't quite understand your travel details. Could you rephrase?")
            continue  # Ask the user to clarify

        print("Debug: Final Captured Travel Details -", travel_details)  # Debug print

        conversation.append({"role": "user", "content": user_input})

        confirmation = intent_confirmation_layer(travel_details, conversation)

        if confirmation["result"] == "No":
            print(f"Chatbot: {confirmation['assistant_response']}")
            continue

        print("Chatbot: Your travel details have been successfully recorded!")

        travel_options = find_matching_travel_options(travel_details, csv_file)
        print("Chatbot: Here are the best travel options for you:")
        print(travel_options)

# Entry point of the script
if __name__ == "__main__":
    main()


Welcome to the India Travel Planner Chatbot!
Tell me about your trip, and I'll take care of the rest.
You: I want to travel to delhi from mumbai via flight in economy class
Debug: Extracted Travel Details - {'Travel_From': 'Mumbai', 'Travel_To': 'Delhi', 'Travel_Mode': 'Flight', 'Class': 'Economy'}
Debug: Final Captured Travel Details - {'Travel_From': 'Mumbai', 'Travel_To': 'Delhi', 'Travel_Mode': 'Flight', 'Class': 'Economy'}
Chatbot: Your travel details have been successfully recorded!
Chatbot: Here are the best travel options for you:
+-----+---------------+-------------+---------------+----------+---------+
|     | Travel_From   | Travel_To   | Travel_Mode   | Class    |   Price |
+=====+===============+=============+===============+==========+=========+
| 216 | Mumbai        | Delhi       | Flight        | Economy  |    2677 |
+-----+---------------+-------------+---------------+----------+---------+
| 321 | Mumbai        | Delhi       | Flight        | Business |    5580 |
+----

KeyboardInterrupt: Interrupted by user